In [145]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import pandas as pd

In [209]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")

def int_to_dummies(i):
    # Create a label for the number given
    #Input: int The value of the label
    result = [0]*30
    result[int(i)] = 1
    return np.array(result)



def prediction_logits (y,batch_size,nb_label):
    # Normalize the batch of label to obtain a percentage
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    #Out : Numpy array [batch_size][10] the input normalized
    
    y_sum = np.sum(y, axis=1)
    for i in range (0,batch_size):
        for j in range (0,nb_label):
            if (y_sum[i] != 0):
                y[i][j] = y[i][j]/y_sum[i]
    
    return y


In [210]:
df = pd.read_csv("data.csv")
df = df[df.Result != -1]
target = df.pop("Result")

In [211]:
df = df.drop(columns = df.columns[0])
df["AT"] = df["AT"].apply(int_to_dummies)
df["HT"] = df["HT"].apply(int_to_dummies)


KeyError: 'AT0'

In [227]:
for i in range(30):
        df["AT"+str(i)] = df["AT"]
        df["HT"+str(i)] = df["HT"]
for k in df.index.values:
    for i in range(30):
        df["AT"+str(i)][k] = df["AT"][k][i]
        df["HT"+str(i)][k] = df["HT"][k][i]
df.pop("AT")
df.pop("HT")

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
                              ...                        
2063    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2064    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2065    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2066    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2067    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
Name: HT, Length: 2067, dtype: object

In [228]:
df

,AT0,HT0,AT1,HT1,AT2,HT2,AT3,HT3,AT4,HT4,...,AT25,HT25,AT26,HT26,AT27,HT27,AT28,HT28,AT29,HT29
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2064,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2065,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
x_train = df.values
y_train = target.values

In [236]:
x_train = x_train.astype(int)

In [238]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Model is the full model w/o custom layers
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train,y_train, epochs=10)
loss, acc = model.evaluate(x_train,y_train)

print("Loss {}, Accuracy {}".format(loss, acc))

Train on 2067 samples
Epoch 1/10
2067/2067 [==============================] - 0s 167us/sample - loss: 1.0965 - accuracy: 0.3769
Epoch 2/10
2067/2067 [==============================] - 0s 42us/sample - loss: 1.0584 - accuracy: 0.4654
Epoch 3/10
2067/2067 [==============================] - 0s 36us/sample - loss: 1.0371 - accuracy: 0.4741
Epoch 4/10
2067/2067 [==============================] - 0s 41us/sample - loss: 1.0144 - accuracy: 0.4983
Epoch 5/10
2067/2067 [==============================] - 0s 40us/sample - loss: 0.9923 - accuracy: 0.5239
Epoch 6/10
2067/2067 [==============================] - 0s 38us/sample - loss: 0.9767 - accuracy: 0.5360
Epoch 7/10
2067/2067 [==============================] - 0s 37us/sample - loss: 0.9667 - accuracy: 0.5375
Epoch 8/10
2067/2067 [==============================] - 0s 43us/sample - loss: 0.9624 - accuracy: 0.5433
Epoch 9/10
2067/2067 [==============================] - ETA: 0s - loss: 0.9465 - accuracy: 0.54 - 0s 47us/sample - loss: 0.9536 - accurac